<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model" data-toc-modified-id="Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Predict" data-toc-modified-id="Predict-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Predict</a></span></li></ul></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="#Voxelize-Mesh" data-toc-modified-id="Voxelize-Mesh-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Voxelize Mesh</a></span></li><li><span><a href="#Sample-SDF" data-toc-modified-id="Sample-SDF-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sample SDF</a></span></li></ul></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys
from pathlib import Path

%load_ext autoreload
%autoreload 2

from DeepSDF import DeepSDF

# Model

In [ ]:
# model config
deep_sdf_config = {
    "nb_layers": 8,
    "latent_dim": 1,
    "inner_activation": "relu",
    "out_activation": "tanh",
    "hidden_dims" : [512]*7,
    "norm_layers" : [0, 1, 2, 3, 4, 5, 6, 7],
    "weight_norm" : True,
    "clamp_delta": 0.1,
}
latent_dim = deep_sdf_config['latent_dim']

In [ ]:
# init model
deep_sdf = DeepSDF(**deep_sdf_config)

In [ ]:
model = deep_sdf.model
model.summary()

In [ ]:
# setup training data
latent = np.random.rand(latent_dim)

#x_train = [all_xyz, np.tile(latent, (all_xyz.shape[0], 1))]
#y_train = mesh_voxels.flatten()
x_train = [points, np.tile(latent, (points.shape[0], 1))]
y_train = sdf

#print(x_train.shape)
print(y_train.shape)

In [ ]:
# train
deep_sdf.train(
    x_train, y_train,
    nb_epochs=10,
    batch_size=256,
    latent=latent
)

## Predict

In [ ]:
pred_voxel = deep_sdf.model.predict([all_xyz, np.tile(latent, (all_xyz.shape[0], 1))])
pred_voxel.shape

In [ ]:
pred_voxel

# Data Processing

In [ ]:
data_dir = Path.home() / 'Documents/generated_data/neural_rendering'

In [ ]:
from mesh_to_sdf import mesh_to_voxels, sample_sdf_near_surface

import trimesh
import skimage.measure

## Voxelize Mesh

In [ ]:
voxel_resolution = 50
mesh = trimesh.load(data_dir / 'suzanne.obj')

mesh_voxels = mesh_to_voxels(mesh, voxel_resolution, pad=False)
mesh_voxels.shape

In [ ]:
voxels = pred_voxel.reshape(tuple([size]*3))

In [ ]:
# show mesh
vertices, faces, normals, _ = skimage.measure.marching_cubes_lewiner(voxels, level=0)
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, vertex_normals=normals)
mesh.show()

In [ ]:
# save voxels to file
np.save(data_dir / 'suzanne.npy', np.array(voxels, dtype=np.float16))

## Sample SDF

In [ ]:
mesh = trimesh.load(data_dir / 'suzanne.obj')

number_of_points = 250000
points, sdf = sample_sdf_near_surface(mesh, number_of_points=number_of_points)
points.shape